In [1]:


from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
#openai_api_base = "http://localhost:8080/v1"
openai_api_base="http://0.0.0.0:8000/v1"
client = OpenAI(
    base_url=openai_api_base,
)

model_name="AXCXEPT/EZO-Common-9B-gemma-2-it"
model_name="weblab-GENIAC/Tanuki-8x8B-dpo-v1.0"
model_name="cyberagent/calm3-22b-chat"


In [2]:

#元の文章のparse
neko_path="data/neko.txt"
with open(neko_path,"r") as f:
    text=f.read()

#clean
lines=text.split("\n")
new_lines=[]
for line in lines:
    line=line.strip()
    if len(line)<10:
        continue
    new_lines.append(line)

len(new_lines)

2105

In [3]:

import random


genres=[
"心温まる日常会話",
"楽しい会話",
"ユーモラスな会話",
"日常会話",
"傾聴の会話",
"相手の興味をそそる会話",
]
def clean_output(text):
    if text.startswith("「"):
        text=text[1:]
    if text.endswith("」"):
        text=text[:-1]
    return text

def ask_model(messages,client):
    completion = client.chat.completions.create(model=model_name,
                                                messages=messages,
                                                temperature=0.7,
                                                max_tokens=1024,
                                                #skip_special_tokens=False,
                                                stop="### 指示:",
                                                )
    text = completion.choices[0].message.content.strip()
    text=clean_output(text)
    return text
def gen_init_text(line,genre):
    system_message=f"""
    次の文章をもとに､{genre}のはじめのセリフを出力しなさい｡対話はタメ口で行われ､敬語は使いません｡セリフのみを出力し、それ以外は何も出力しないでください｡
    """


    prompt=f"""
    #参考にする文章: {line}
    """

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
                ]
    text=ask_model(messages,client)
    return text

In [4]:
n_replies=4
def gen_text(new_lines,genres,client,
        n_replies=8):
    line=random.choice(new_lines)
    genre=random.choice(genres)



    text=gen_init_text(line,genre)

    system_message=f"""{genre}の返答文を生成しなさい｡対話はタメ口で行われ､敬語は使いません｡セリフのみを出力し、それ以外は何も出力しないでください｡"""
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": text}
        ]

    for j in range(n_replies):

        res=ask_model(messages,client)
        if j%2==1:
            messages.append({"role": "user", "content": res})
        else:
            messages.append({"role": "assistant", "content": res})

    return messages

In [5]:
messages=gen_text(new_lines,genres,client,n_replies=7)
messages

[{'role': 'system',
  'content': 'ユーモラスな会話の返答文を生成しなさい｡対話はタメ口で行われ､敬語は使いません｡セリフのみを出力し、それ以外は何も出力しないでください｡'},
 {'role': 'user',
  'content': 'お前、また蕎麦の話か？この前も饂飩の話してたじゃん。そんなに蕎麦が好きなら、蕎麦屋でも始めれば？'},
 {'role': 'assistant',
  'content': '蕎麦屋かー、それもいいかもな！でも、まずは美味しい蕎麦が打てるようにならなきゃな。毎日食べても飽きないくらいの美味しさを目指したいね。'},
 {'role': 'user',
  'content': '毎日蕎麦でも全然飽きないとか、どんだけ好きなんだよ！まぁ、でも確かにソバってマジで奥が深いよな。打ち方一つで全然違う味になるし。'},
 {'role': 'assistant',
  'content': 'そうそう！温度とか湿度でも全然違うんだよ。だから、毎回同じ味にするのが大変なんだ。でも、その分、上手くいった時の感動はすごいんだよね。'},
 {'role': 'user',
  'content': 'わかるわー。俺も試しに打ってみたけど、全然ダメだった。まあ、練習あるのみだよね。でも、お前が打った蕎麦、一度食べてみたいな。'},
 {'role': 'assistant',
  'content': 'おお、それは楽しみにしててくれ！美味しい蕎麦打てるように、毎日練習してるからさ。一緒に蕎麦パーティーとかどう？'},
 {'role': 'user',
  'content': 'それ、いいじゃん！じゃあ、週末にでもやってみようぜ。みんなで集まってワイワイやるのも楽しそうだし。'},
 {'role': 'assistant',
  'content': '最高だな！じゃあ、材料とか準備しておくから、みんなに連絡しといてよ。美味しい蕎麦と、あとは何食べようか？'}]

In [6]:

import json
from tqdm import tqdm
import time
out_file_path="data/neko_gen_mult.jsonl"

with open(out_file_path,"a") as f:
    for i in tqdm(range(10**4)):
        messages=gen_text(new_lines,genres,client,n_replies=7)
        f.write(json.dumps({"messages":messages},ensure_ascii=False)+"\n")
        time.sleep(1)

  0%|          | 9/10000 [01:07<20:55:14,  7.54s/it]


KeyboardInterrupt: 